In [1]:
import numpy as np
import pandas as pd

from scipy.stats import chi2
from scipy.optimize import curve_fit
from scipy.optimize import minimize

import plotly.graph_objects as go

In [2]:
data = pd.read_csv('/content/Rotcurve_Impares_errores.dat',
                   names = ['R', 'V', 'epsilon'], sep = '\s+')

In [3]:
G = 43007.1

# UM, UL, Ut, UV =  1e10, 1e10, 1.1e9, 1

mu_epsilon = data.epsilon.mean()
sigma_epsilon = data.epsilon.std()

In [104]:
disk_velocity = lambda R, Md: G*Md*R/(R**2 + 7.5**2)**(3/2)

halo_velocity1 = lambda R, Mh, rs: G*Mh*R/(R + rs)**2

halo_velocity2 = lambda R, Mh, rs: G*Mh/(R + rs)

def circular_velocity1(R, Md, Mh, rs):

  return (disk_velocity(R, Md) + halo_velocity1(R, Mh, rs))**0.5

def circular_velocity2(R, Md, Mh, rs):

  return (disk_velocity(R, Md) + halo_velocity2(R, Mh, rs))**0.5

def pdf_error(x, func):

  epsilon = data.V - func(data.R, *x)

  return np.exp((-(epsilon-mu_epsilon)**2).sum()/(2*sigma_epsilon**2))

def chisq(x, func):

  Vc = func(data['R'], *x)

  return np.sum((data['V'] - Vc)**2 / Vc)

In [ ]:
Mds = np.linspace(90,150, 50)
Mhs = np.linspace(0,1,50)
rss = np.linspace(0,0.8,50)

vals = [[[pdf_error([x, y ,z], circular_velocity1) for z in rss]
        for y in Mhs] for x in Mds]

Dn = np.trapz(np.trapz(np.trapz(vals, x = rss), x = Mhs), x = Mds)
pdf_Md = np.trapz(np.trapz(vals, x = rss), x = Mhs)/Dn
pdf_Mh = np.trapz(np.trapz(vals, x = rss), x = Mds, axis = 0)/Dn
pdf_rs = np.trapz(np.trapz(vals, axis = 1, x = Mhs), x = Mds, axis = 0)/Dn

In [102]:
# mu_Mds = (Mds*pdf_Md).sum()/pdf_Md.sum()
# mu_Mhs = (Mhs*pdf_Mh).sum()/pdf_Mh.sum()
# mu_rss = (rss*pdf_rs).sum()/pdf_rs.sum()

x1 = np.array([Mds[pdf_Md.argmax()], Mhs[pdf_Mh.argmax()], rss[pdf_rs.argmax()]])
# x1 = np.array([mu_Mds, mu_Mhs, mu_rss])

# sigma_Mds = (((Mds - mu_Mds)**2*pdf_Md).sum()/pdf_Md.sum())**0.5
# sigma_Mhs = (((Mhs - mu_Mhs)**2*pdf_Mh).sum()/pdf_Mh.sum())**0.5
# sigma_rss = (((rss - mu_rss)**2*pdf_rs).sum()/pdf_rs.sum())**0.5

# sigma_x1 = np.array([sigma_Mds, sigma_Mhs, sigma_rss])

In [111]:
go.Figure([go.Scatter(x = data['R'],
                      y = data['V'],
                      name = 'Data',
                      error_y_array = data.epsilon),
           go.Scatter(x = data['R'],
                      y = circular_velocity1(data['R'], *x1),
                      error_y_array = circular_velocity1(data['R'], *x1) - circular_velocity1(data['R'], *(x1 - sigma_x1)),
                      name = '1st Model'),
           go.Scatter(x = data['R'],
                      y = circular_velocity2(data['R'], *x2),
                      error_y_array = circular_velocity2(data['R'], *x2) - circular_velocity2(data['R'], *(x2 - (sigma_x2.diagonal())**0.5)),
                      name = '2nd Model')],

          layout = go.Layout(xaxis_title = 'R [kpc]',
                             yaxis_title = 'Vc [km/s]',
                             height = 600, width = 900,
                             legend = dict(orientation = 'h',
                                           x = 0.6, y = 1.08)))

In [96]:
go.Figure([go.Scatter(x = Mds, y = pdf_Md)],

          layout = go.Layout(xaxis_title = 'Disk Mass [1e10 M⊙]',
                             yaxis_title = 'Probability Density',
                             height = 600, width = 900,
                             legend = dict(orientation = 'h',
                                           x = 0.85, y = 1.08)))

In [95]:
go.Figure([go.Scatter(x = Mhs, y = pdf_Mh)],

          layout = go.Layout(xaxis_title = 'Halo Mass [1e10 M⊙]',
                             yaxis_title = 'Probability Density',
                             height = 600, width = 900,
                             legend = dict(orientation = 'h',
                                           x = 0.85, y = 1.08)))

In [109]:
x2, (sigma_x2.diagonal())**0.5

(array([1.08414139e+02, 1.60299096e+08, 5.78778350e+08]),
 array([1.54632224e+01, 4.49768509e+14, 1.62394102e+15]))

In [92]:
go.Figure([go.Scatter(x = rss, y = pdf_rs)],

          layout = go.Layout(xaxis_title = 'Halo Size Scale [kpc]',
                             yaxis_title = 'Probability Density',
                             height = 600, width = 900,
                             legend = dict(orientation = 'h',
                                           x = 0.85, y = 1.08)))



---



In [69]:
x2, sigma_x2 = curve_fit(circular_velocity2, data['R'], data['V'])

chisq1 = chisq(x1, circular_velocity1)
chisq2 = chisq(x2, circular_velocity2)

In [80]:
chi2.sf(chisq1, df = len(data.V) + 2), chi2.sf(chisq2, df = len(data.V) + 2)

(3.969637283776825e-24, 1.1682570573489784e-16)



---

